Ensuring that your dataset is stored as efficiently as possible is crucial for allowing models to run on a more reasonable timescale without having to reduce the size of the dataset.

The task is to clean up one of the companies largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [2]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [3]:
# Get the info and memory usage on the ds_jobs dataset
print(ds_jobs.memory_usage())

print(ds_jobs.info())

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolle

### In the next cell, I will be doing a value_count of each column one at a time and assess what way can it be stored in order to save memory.

In [50]:
## Count the values of object type columns
ds_jobs['enrolled_university'].value_counts()

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

### Create a copy of the df and transforming data

In [4]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Changing all the int64 columns to int32
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype("int32")

ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype("int32")

# Changing all the categories with two factors as booleans
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype("bool")

ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype("bool")

# Changing all the float64 columns to float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype("float16")

# Changing object columns containing nominal categorical data to category type

ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype("category")

ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype("category")

ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype("category")

ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype("category")

In [5]:
# Changing object columns containing ordinal categorical data to category type
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype("category")

ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype("category")

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype("category")

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype("category")

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype("category")

# Reorder categories into a natural order
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.reorder_categories(
    new_categories = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    ordered = True)

ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].cat.reorder_categories(
    new_categories = ['no_enrollment', 'Part time course', 'Full time course'],
    ordered = True)

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].cat.reorder_categories(
    new_categories = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
                      '14', '15', '16', '17', '18', '19', '20', '>20'],
    ordered = True)

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].cat.reorder_categories(
    new_categories = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    ordered = True)

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(
    new_categories = ['never', '1', '2', '3', '4', '>4'],
    ordered = True)

In [6]:
# Filter for students with 10 or more years of experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['experience'] >= "10"]
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['company_size'] >= "1000-4999"]

### Compare the memory usage from the original dataset

In [9]:
print(ds_jobs.memory_usage())
print()
print(ds_jobs_transformed.memory_usage())

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 2201
dtype: int6